# TITLE ![image text](img/International_Pokémon_logo.svg)
## Subtitle
Intro Text

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

uri = os.getenv("MONGO_URI")

print("URI scheme:", (uri or "MISSING").split("://", 1)[0])

# Use public DNS resolvers instead of Proton's
import dns.resolver
r = dns.resolver.Resolver(configure=False)
r.nameservers = ["1.1.1.1", "8.8.8.8"]  # Cloudflare + Google
dns.resolver.default_resolver = r

import certifi
from pymongo import MongoClient

client = MongoClient(
    uri,
    tls=True,
    tlsCAFile=certifi.where(),
    serverSelectionTimeoutMS=30000,
    uuidRepresentation="standard",
)
print("Pinging admin...")
client.admin.command("ping")
print("✅ Connected to MongoDB Atlas")


URI scheme: mongodb+srv
Pinging admin...


ServerSelectionTimeoutError: SSL handshake failed: ac-wy9gmwk-shard-00-01.zwxbxnm.mongodb.net:27017: [WinError 10054] An existing connection was forcibly closed by the remote host (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms),SSL handshake failed: ac-wy9gmwk-shard-00-02.zwxbxnm.mongodb.net:27017: [WinError 10054] An existing connection was forcibly closed by the remote host (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms),SSL handshake failed: ac-wy9gmwk-shard-00-00.zwxbxnm.mongodb.net:27017: [WinError 10054] An existing connection was forcibly closed by the remote host (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30.0s, Topology Description: <TopologyDescription id: 68fb50f699f12783fdc7d309, topology_type: ReplicaSetNoPrimary, servers: [<ServerDescription ('ac-wy9gmwk-shard-00-00.zwxbxnm.mongodb.net', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('SSL handshake fail